In [3]:
from IPython.display import HTML, display

html = r"""
<style>
:root{
  --card-size: 90px;
  --gap: 10px;
  --bg:#0f172a;
  --card-back:#1e293b;
  --accent:#f97316;
  --panel:#0b1220;
  --text:#e6eef8;
}
*{box-sizing:border-box;font-family:Inter, system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", Arial;}
body{background:var(--bg); color:var(--text); padding:18px;}
.game-wrap{max-width:880px;margin:12px auto;background:linear-gradient(180deg, rgba(255,255,255,0.02), transparent);padding:16px;border-radius:12px;box-shadow:0 6px 30px rgba(2,6,23,0.6);}
.header{display:flex;gap:12px;align-items:center;justify-content:space-between;margin-bottom:12px}
.title{font-size:20px;font-weight:600;display:flex;gap:10px;align-items:center}
.controls{display:flex;gap:8px;align-items:center}
.btn{background:var(--panel);border:1px solid rgba(255,255,255,0.03);padding:8px 12px;border-radius:8px;color:var(--text);cursor:pointer}
.select{background:transparent;padding:6px;border-radius:8px;border:1px solid rgba(255,255,255,0.04);color:var(--text)}
.stats{display:flex;gap:12px;align-items:center;font-size:14px}
.board{display:grid;gap:var(--gap);justify-content:center;padding:10px}
.card{width:var(--card-size);height:var(--card-size);perspective:800px;cursor:pointer}
.card-inner{position:relative;width:100%;height:100%;transition:transform .45s;transform-style:preserve-3d;border-radius:10px}
.card.flipped .card-inner{transform:rotateY(180deg)}
.face{position:absolute;inset:0;border-radius:10px;display:flex;align-items:center;justify-content:center;font-size:36px}
.front{background:linear-gradient(180deg, #111827, #0b1220);transform:rotateY(180deg);border:1px solid rgba(255,255,255,0.03)}
.back{background:var(--card-back);border:1px solid rgba(255,255,255,0.02);color:#cbd5e1;font-size:20px}
.matched{opacity:.35;filter:grayscale(1);pointer-events:none}
.timer{width:160px;height:12px;background:rgba(255,255,255,0.05);border-radius:8px;overflow:hidden}
.timer > i{display:block;height:100%;background:linear-gradient(90deg,var(--accent),#ef4444);width:100%}
.footer{display:flex;justify-content:space-between;align-items:center;margin-top:12px}
.center{display:flex;gap:8px;align-items:center}
.modal{position:fixed;left:0;right:0;top:0;bottom:0;background:rgba(2,6,23,0.6);display:flex;align-items:center;justify-content:center;z-index:9999}
.dialog{background:#071024;padding:20px;border-radius:12px;min-width:280px;text-align:center;border:1px solid rgba(255,255,255,0.03)}
.small{font-size:13px;color:#9fb0d6}
.logo{width:36px;height:36px;background:linear-gradient(135deg,#f97316,#f43f5e);display:inline-flex;align-items:center;justify-content:center;border-radius:9px;color:white;font-weight:700}
.note{font-size:13px;color:#9fb0d6;margin-top:6px}
</style>

<div class="game-wrap">
  <div class="header">
    <div class="title"><div class="logo">M</div> Memory Puzzle — Colab Edition</div>
    <div class="controls">
      <label class="small">Grid:</label>
      <select id="gridSelect" class="select">
        <option value="4">4x4 (8 pairs)</option>
        <option value="6">6x6 (18 pairs)</option>
        <option value="2">2x2 (2 pairs)</option>
      </select>
      <label class="small">Time (sec):</label>
      <select id="timeSelect" class="select">
        <option>60</option>
        <option selected>90</option>
        <option>120</option>
        <option>30</option>
      </select>
      <button id="restartBtn" class="btn">Restart</button>
    </div>
  </div>

  <div class="stats">
    <div class="center"><strong>Moves:</strong>&nbsp;<span id="moves">0</span></div>
    <div class="center"><strong>Pairs:</strong>&nbsp;<span id="pairs">0</span>/<span id="totalPairs">0</span></div>
    <div style="flex:1"></div>
    <div style="width:220px">
      <div style="display:flex;justify-content:space-between;font-size:13px"><span>Time left</span><span id="timeLabel">--</span></div>
      <div class="timer" title="time left"><i id="timeBar" style="width:100%"></i></div>
    </div>
  </div>

  <div id="board" class="board" style="margin-top:12px"></div>

  <div class="footer">
    <div class="note">Tip: Click cards to flip and match pairs before time runs out!</div>
    <div><button id="hintBtn" class="btn">Hint (reveal 1 pair)</button></div>
  </div>
</div>

<div id="endModal" style="display:none" class="modal">
  <div class="dialog">
    <h3 id="endTitle">You did it!</h3>
    <div id="endStats" class="small"></div>
    <div style="margin-top:12px">
      <button id="playAgain" class="btn">Play again</button>
    </div>
  </div>
</div>

<script>
(() => {
  const emojis = ["🍎","🍊","🍋","🍇","🍓","🍉","🍒","🥝","🍍","🥭","🍑","🥥","🍐","🍈","🍅","🥕","🌽","🍆","🥔","🥦","🌶️","🧄","🧅","🥬","🫐","🍌","🍠","🍯","🥨","🧇"];
  const boardEl = document.getElementById('board');
  const movesEl = document.getElementById('moves');
  const pairsEl = document.getElementById('pairs');
  const totalPairsEl = document.getElementById('totalPairs');
  const timeLabel = document.getElementById('timeLabel');
  const timeBar = document.getElementById('timeBar');
  const endModal = document.getElementById('endModal');
  const endTitle = document.getElementById('endTitle');
  const endStats = document.getElementById('endStats');
  const playAgain = document.getElementById('playAgain');
  const restartBtn = document.getElementById('restartBtn');
  const gridSelect = document.getElementById('gridSelect');
  const timeSelect = document.getElementById('timeSelect');
  const hintBtn = document.getElementById('hintBtn');

  let gridN = parseInt(gridSelect.value);
  let totalPairs = (gridN*gridN)/2;
  let timeLimit = parseInt(timeSelect.value);
  let timerInterval = null;
  let timeLeft = timeLimit;
  let first=null, second=null;
  let moves=0, matchedPairs=0;
  let lock=false;

  function shuffle(a){
    for(let i=a.length-1;i>0;i--){
      const j=Math.floor(Math.random()*(i+1));
      [a[i],a[j]]=[a[j],a[i]];
    }
    return a;
  }

  function buildBoard(){
    clearInterval(timerInterval);
    gridN = parseInt(gridSelect.value);
    timeLimit = parseInt(timeSelect.value);
    timeLeft = timeLimit;
    moves=0; matchedPairs=0; first=null; second=null; lock=false;
    movesEl.textContent = moves;
    pairsEl.textContent = matchedPairs;
    totalPairs = (gridN*gridN)/2;
    totalPairsEl.textContent = totalPairs;
    timeLabel.textContent = timeLeft;
    timeBar.style.width = '100%';
    document.documentElement.style.setProperty('--card-size', gridN <= 4 ? '90px' : (gridN===6?'70px':'120px'));
    boardEl.style.gridTemplateColumns = `repeat(${gridN}, var(--card-size))`;

    const copy = emojis.slice(0);
    shuffle(copy);
    const chosen = copy.slice(0, totalPairs);
    const deck = shuffle(chosen.concat(chosen).slice());
    boardEl.innerHTML = '';
    deck.forEach((sym, idx) => {
      const c = document.createElement('div');
      c.className='card';
      c.dataset.val = sym;
      c.dataset.idx = idx;
      c.innerHTML = `<div class="card-inner"><div class="face back">?</div><div class="face front">${sym}</div></div>`;
      c.addEventListener('click', () => onCardClick(c));
      boardEl.appendChild(c);
    });

    timerInterval = setInterval(() => {
      timeLeft--;
      timeLabel.textContent = timeLeft;
      const pct = Math.max(0, (timeLeft/timeLimit)*100);
      timeBar.style.width = pct + '%';
      if(timeLeft<=0){
        clearInterval(timerInterval);
        finish(false, 'Time\'s up!');
      }
    }, 1000);
  }

  function onCardClick(card){
    if(lock) return;
    if(card.classList.contains('flipped') || card.classList.contains('matched')) return;
    card.classList.add('flipped');
    if(!first){
      first = card;
      return;
    }
    if(card === first) return;
    second = card;
    lock = true;
    moves++;
    movesEl.textContent = moves;
    if(first.dataset.val === second.dataset.val){
      setTimeout(() => {
        first.classList.add('matched');
        second.classList.add('matched');
        matchedPairs++;
        pairsEl.textContent = matchedPairs;
        first=null; second=null; lock=false;
        if(matchedPairs>=totalPairs){
          clearInterval(timerInterval);
          finish(true, 'You matched all pairs!');
        }
      }, 450);
    } else {
      setTimeout(() => {
        first.classList.remove('flipped');
        second.classList.remove('flipped');
        first=null; second=null; lock=false;
      }, 700);
    }
  }

  function finish(win, msg){
    endTitle.textContent = win ? "🎉 You Win!" : "⏰ Game Over";
    endStats.innerHTML = `<div>${msg}</div><div>Moves: ${moves} | Pairs: ${matchedPairs}/${totalPairs}</div>`;
    endModal.style.display='flex';
  }

  playAgain.addEventListener('click', ()=>{
    endModal.style.display='none';
    buildBoard();
  });

  restartBtn.addEventListener('click', ()=>{ endModal.style.display='none'; buildBoard(); });
  gridSelect.addEventListener('change', ()=> buildBoard());
  timeSelect.addEventListener('change', ()=> buildBoard());

  hintBtn.addEventListener('click', ()=>{
    const unmatched = Array.from(boardEl.querySelectorAll('.card:not(.matched):not(.flipped)'));
    if(unmatched.length<2) return;
    const map = {};
    for(const c of unmatched){
      const v = c.dataset.val;
      if(!map[v]) map[v] = [];
      map[v].push(c);
    }
    let pair = null;
    for(const k in map){
      if(map[k].length>=2){ pair = map[k].slice(0,2); break; }
    }
    if(!pair){
      pair = unmatched.slice(0,2);
    }
    pair.forEach(c=>c.classList.add('flipped'));
    setTimeout(()=> pair.forEach(c=>c.classList.remove('flipped')), 900);
  });

  buildBoard();
})();
</script>
"""

display(HTML(html))
